The NOAA HRRR is a real-time 3km resolution, hourly updated, cloud-resolving, convection-allowing atmospheric model, initialized by 3km grids with 3km radar assimilation.

This notebook provides an example of accessing HRRR data, including (1) finding the data file corresponding to a date and time, (2) retrieving a portion of that file from blob storage which includes the surface temperature variable, (3) opening the file using the xarray library, and (4) rendering an image of the forecast.

This dataset is documented at http://aka.ms/ai4edata-hrrr.

In [3]:
import climatedata_functions as climf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
metadata = pd.read_csv("../data/metadata.csv")

In [ ]:
ds = climf.get_ds()#only words with recent dates

### Make metadafile with gridpoints

In [ ]:
#tranform coordinates for longitude
metadata['longitude_trans'] = metadata['longitude']+360 
#add columns for gridpoints
metadata['x_grid'] = ''
metadata['y_grid'] = ''

#make new metadatafile and save as cvs 
#metadata_new = climf.save_grids(metadata.head(), ds) #(takes approx 6 minutes)
#metadata_new.to_csv('../data/metadata_grids.csv')


In [4]:
#load new metadata file that includes the gridpoints
metadata = pd.read_csv("../data/metadata_grids.csv")

In [7]:
metadata.head(3)

,Unnamed: 0,uid,latitude,longitude,date,split,longitude_trans,x_grid,y_grid
0,0,aabm,39.080319,-86.430867,2018-05-14,train,273.569133,570,1217
1,1,aabn,36.559700,-121.510000,2016-08-31,test,238.490000,550,192
2,2,aacd,35.875083,-78.878434,2020-11-19,train,281.121566,488,1455


### Get temperatures for gridpoints

Make function that gets the mean/median temperature for one date for all the places sampled at that date (to save time) and the previos x days

In [9]:
metadata = pd.read_csv("../data/metadata_temp_12_01_06_complete_without_a.csv")

In [10]:
from datetime import timedelta
import warnings
warnings.simplefilter(action='ignore')

#make empty list to store the dates already sampled
done_list = []

#define important download parameters
days = 7
hour = 1 #which our of the day, impoetant: also change in the columnnames 2 times!!!)
metadata['temp_01'] = '' #create new column to dstore the data #change also in last line before the else!!!

#getting the data
for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#only do if this date wasnt used before
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        #print(len(temp_list), len(temp_list[0]))
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds, stop = climf.get_ds_aws(day_date, hour)#getting the temperature array for the specified date
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = np.nan
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata.temp_01.loc[date_index_list[i]] = temp_list[i] #not index but  
    else:
        continue 

2018-05-14


In [ ]:
metadata.to_csv('../data/metadata_temp_01_06_12_complete.csv')

If the download was interupted the code in the next cell can be used to continue with the download. Since the dates already dowloaded are saved in 'done_list', it can continue from that withouth starting from scratch.

In [ ]:
done_list.pop()#remove last element (in case it didn work with that)
len(done_list)#how many dates are already inside (1637 unique dates in total)

In [ ]:
#
import warnings
warnings.simplefilter(action='ignore')


temp_list = []
#done_list = [] deactivate because we want to continue from the old donelist
days = 7
hour = 12 #which our of the day


for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#only do if this date wasnt used before
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        #print(len(temp_list), len(temp_list[0]))
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds, stop = climf.get_ds_aws(day_date, hour)#getting the temperature array for the specified date
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = np.nan
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata.temp_01.loc[date_index_list[i]] = temp_list[i] #not index but  
    else:
        continue 

In [ ]:
#metadata.to_csv('../data/metadata_temp_12_01_06_complete.csv')